# DSCI 417 - Homework 08
**Malcolm Nichols**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()

## Problem 1: Creating Streaming DataFrame

In [0]:
static_df = (
  spark.read
  .option('header', True)
  .option('inferSchema', True)
  .csv('/FileStore/tables/paysim/step_001.csv')
)

paysim_schema = static_df.schema

static_df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
step| type| amount| nameOrig|oldbalanceOrg|newbalanceOrig| nameDest|oldbalanceDest|newbalanceDest|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
 1| PAYMENT| 9839.64|C1231006815| 170136.0| 160296.36|M1979787155| 0.0| 0.0|
 1| PAYMENT| 1864.28|C1666544295| 21249.0| 19384.72|M2044282225| 0.0| 0.0|
 1|TRANSFER| 181.0|C1305486145| 181.0| 0.0| C553264065| 0.0| 0.0|
 1|CASH_OUT| 181.0| C840083671| 181.0| 0.0| C38997010| 21182.0| 0.0|
 1| PAYMENT|11668.14|C2048537720| 41554.0| 29885.86|M1230701703| 0.0| 0.0|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
only showing top 5 rows

In [0]:
paysim_stream = (
    spark.readStream
    .option('header', True)
    .option('maxFilesPerTrigger', 1)
    .schema(paysim_schema)
    .csv('/FileStore/tables/paysim/')
)

print(paysim_stream.isStreaming)

True

## Problem 2: Apply Transformations

In [0]:
type_summary = (
    paysim_stream
    .groupBy('type')
    .agg(
        expr('COUNT(*) AS n'),
        expr('ROUND(MEAN(amount),2) AS avg_amount'),
        expr('MIN(amount) AS min_amount'),
        expr('MAX(amount) AS max_amount')
    )
    .sort('n', ascending=False)
)

In [0]:
destinations = (
    paysim_stream
    .filter(expr('type == "TRANSFER"'))
    .groupBy('nameDest')
    .agg(
        expr('COUNT(*) AS n'),
        expr('ROUND(SUM(amount),2) AS total_amount'),
        expr('ROUND(MEAN(amount),2) AS avg_amount')
    )
    .sort('n', ascending=False)
)

## Problem 3: Define Output Sinks

In [0]:
type_writer = (
    type_summary
    .writeStream
    .format('memory')
    .queryName('type_summary')
    .outputMode('complete')
)

dest_writer = (
    destinations
    .writeStream
    .format('memory')
    .queryName('destinations')
    .outputMode('complete')
)

## Problem 4: Start and Monitor the Streams

In [0]:
type_query = type_writer.start()
dest_query = dest_writer.start()

In [0]:
spark.sql('SELECT * FROM type_summary').show()

+--------+------+----------+----------+----------+
 type| n|avg_amount|min_amount|max_amount|
+--------+------+----------+----------+----------+
CASH_OUT|283602| 187343.61| 0.37| 1.0E7|
 PAYMENT|262254| 11618.97| 0.1| 115264.68|
 CASH_IN|172327| 172399.61| 5.44|1781905.26|
TRANSFER| 65121| 665790.8| 2.6| 1.0E7|
 DEBIT| 5365| 5846.53| 0.87| 218148.28|
+--------+------+----------+----------+----------+

In [0]:
print(spark.sql('SELECT * FROM destinations').count())
spark.sql('SELECT * FROM destinations').show(16)

30875
+-----------+---+-------------+----------+
 nameDest| n| total_amount|avg_amount|
+-----------+---+-------------+----------+
 C665576141| 26| 2.61501039E7|1005773.23|
C1286084959| 26| 1.75882591E7| 676471.5|
 C248609774| 24| 2.47118628E7|1029660.95|
 C306206744| 22|1.304888507E7| 593131.14|
 C985934102| 22|1.441611937E7| 655278.15|
C1360767589| 22|2.175636292E7| 988925.59|
C1590550415| 20|2.635039592E7| 1317519.8|
 C97730845| 20|2.800987886E7|1400493.94|
 C451111351| 20|1.978348034E7| 989174.02|
 C716083600| 19|2.026890477E7|1066784.46|
C1883840933| 19|2.386476521E7|1256040.27|
 C863811613| 18|2.186886699E7|1214937.06|
C2083562754| 18|2.198921771E7|1221623.21|
 C803116137| 17|1.864215501E7|1096597.35|
C2011996123| 16|1.459595107E7| 912246.94|
C1255024717| 16|1.451180338E7| 906987.71|
+-----------+---+-------------+----------+
only showing top 16 rows

In [0]:
type_query.stop()
dest_query.stop()